In [ ]:
import ray
import torch
from rllib_setup import get_env_continuous

In [ ]:
env = get_env_continuous()
env.action_space

In [ ]:
ray.init()

In [ ]:
env = get_env()

In [ ]:
env_name = "VJS"

In [ ]:
ray.rllib.utils.check_env(env)

In [ ]:
from ray.tune import register_env
register_env(env_name,lambda config: get_env())

In [ ]:
from ray.rllib.policy.policy import PolicySpec
test_env = get_env()
obs_space = test_env.observation_space
act_space = test_env.action_space
def policies(agent_ids):
    return {
        str(i): PolicySpec(
            # observation_space=obs_space,
            # action_space=act_space,
            config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }

In [ ]:
action = test_env.action_space_sample()
test_env.step(action)

In [ ]:
test_env.action_space

In [ ]:
from ray.tune.registry import get_trainable_cls
import os
config = (
    get_trainable_cls("MADDPG")
    .get_default_config()
    .environment(env=env_name)
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
)
(
    config.framework("tf")
    # .environment(env_config={"actions_are_logits": True})
    .training(num_steps_sampled_before_learning_starts=100)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: str(
            agent_id
        ),
    )
)


In [ ]:
%load_ext rich

In [ ]:
from ray import air, tune
stop = {
    "training_iteration": 10000,
}
results = tune.Tuner(
    "MADDPG",
    run_config=air.RunConfig(stop=stop, verbose=2),
    param_space=config,
).fit()

In [ ]:
import ray
from ray.rllib.algorithms.ddpg.ddpg import DDPGConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env
import rich  
import rllib_setup

env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): PolicySpec(
            observation_space=obs_space,
            action_space=act_space,
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }




In [ ]:
config = (
    DDPGConfig()
    .training(lr=0.01)
    .resources(num_gpus=1)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
)
# config.batch_mode = "complete_episodes"
rich.print(config.to_dict())

In [ ]:
algo = config.build(env=env_name)

In [ ]:
algo.train()


In [ ]:
from ray import tune
from ray import air

In [ ]:
tune.Tuner(  
    "DDPG",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit()

In [6]:
from ray.rllib.algorithms.sac import SACConfig
from ray.tune import register_env
import rllib_setup

env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }
    
    
config = SACConfig().training(gamma=0.9, lr=0.01)
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=4).multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
config.batch_mode = "complete_episodes"
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env=env_name)  
algo.train()

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': False, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_envs_p

/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:442: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future 

{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'Machine_0': {'learner_stats': {'allreduce_latency': 0.0,
     'grad_gnorm': 1.6580369472503662,
     'actor_loss': -0.6601142287254333,
     'critic_loss': 0.18166503310203552,
     'alpha_loss': 0.0,
     'alpha_value': 1.0,
     'log_alpha_value': 0.0,
     'target_entropy': -1.0,
     'policy_t': 0.15022191405296326,
     'mean_q': 0.008084948174655437,
     'max_q': 0.023381337523460388,
     'min_q': -0.013059914112091064},
    'td_error': array([0.1546048 , 0.6731891 , 0.63759154, 0.36402893, 0.7949961 ,
           0.7985662 , 0.52084607, 0.04691811, 0.7126701 , 0.6651226 ,
           0.7388468 , 0.36920813, 0.4251228 , 0.1735494 , 0.66400844,
           0.80126774, 0.81387985, 0.2162496 , 0.52262676, 0.7517286 ,
           0.85137606, 0.7869222 , 0.74207294, 0.17382504, 0.6364951 ,
           0.2867324 ], dtype=float32),
    'mean_td_error': 0.5508633255958557,
    'model': {},
    'custom_metrics': {},
    'num

In [8]:
from ray.rllib.algorithms.maddpg.maddpg import MADDPGConfig
config = MADDPGConfig()
print(config.replay_buffer_config)  
replay_config = config.replay_buffer_config.update(  
    {
        "capacity": 100000,
        "prioritized_replay_alpha": 0.8,
        "prioritized_replay_beta": 0.45,
        "prioritized_replay_eps": 2e-6,
    }
)
config.training(replay_buffer_config=replay_config)   
config = config.resources(num_gpus=0)   
config = config.rollouts(num_rollout_workers=4)   
config = config.environment(env=env_name)  
algo = config.build()  
algo.train()  

{'type': 'MultiAgentReplayBuffer', 'prioritized_replay': -1, 'capacity': 1000000, 'replay_mode': 'lockstep'}


/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:442: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future 

ValueError: Must set `agent_id` in the policy config.